# Külső API kezelése

API-t akkor használunk ha valamire nincs python könyvtár... csak vicceltem, mindenre van python könyvtár. :-) No, de ha esetleg mégsincs, vagy -- ami valószínűbb -- egyszerűen nincsenek meg lokálisan az adatok, vagy a gépünk teljesítménye nem elegendő, nem tehetünk mást mint külső rendszert veszünk igénybe API hívások segítségével.

Amennyiben ez egy HTTP alapú REST API (mert van azért másmilyen is) akkor a request könyvtár lesz a legjobb barátunk.

Mivel azonban a legtöbb ilyen REST API egy JSON nevű (szöveges) formátumot ad vissza, előbb tekintsük át gyorsan mi is ez.

## JSON

A JSON formátum annyira nem lesz idegen, ugyanis pont olyan (na jó, majdnem) mint a dict és list amit már ismerünk, amiben lehetnek számok, lebegőpontos számok és szövegek, sőt akár még None is, csak azt ott null-nak hívják. Használni sem lesz nehéz, mert természetesen van hozzá programkönyvtár.

In [ ]:
import json

# csináljunk egy adat-vegyesfelvágottat:
python_szerkezet = {
    "szöveg": "ez egy szöveg",
    "számok": [1,2,3,4,5],
    "semmi": None,
    "tuple": (18,42),
    1999: 0.456,
    0: 0
}

# alakítsuk JSON formátumúvá:
json_text = json.dumps(python_szerkezet)
json_text

'{"sz\\u00f6veg": "ez egy sz\\u00f6veg", "sz\\u00e1mok": [1, 2, 3, 4, 5], "semmi": null, "tuple": [18, 42], "1999": 0.456, "0": 0}'

Ez az eredményül kapott szöveg a JSON. Látszik, hogy szinte teljesen úgy néz ki mint a python adat, néhány apró különbséggel (pl, átnevezte a None-t, escapelte az unicode karaktereket, vagy mindig szöveget készít a kulcsokból).

Miért jó ez nekünk? A JSON-t nagyon-nagyon sok nyelv és rendszer érti (szinte mind) így kiváló lehetőség, hogy a struktúrált adatainkat máshová tudjuk vinni.

Mivel a JSON-ben lehet lista és dict is, dict-ben másik dict abban egy lista és így tovább, kifejezetten komplex szerkezeteket le tudunk benne írni.

Ha JSON adatunk van és pythonban szeretnénk dolgozni vele, akkor visszaalakítjuk:

In [ ]:
python_adat = json.loads(json_text)
python_adat

{'szöveg': 'ez egy szöveg',
 'számok': [1, 2, 3, 4, 5],
 'semmi': None,
 'tuple': [18, 42],
 '1999': 0.456,
 '0': 0}

Szinte ugyananaz, mint ahonnan indultunk, csak a kulcsok szövegek lettek, a tuple pedig listává vált (ezeket nem ismeri a JSON). Sajnos akármit azért nem tudunk JSON-é alakítani. Ha extrább python szerkezetünk van, pl. egy halmaz (set) akkor azt nem tudjuk (automatikusan) JSON-é alakítani.

## API hívások: request

Most, hogy már tudjuk mit fogunk visszakapni, nézzük néhány példát hogyan tudjuk ezeket a külső rendszereket használni.

Kezdjük egy pénzváltóval ami az aktuális deviza árfolyamokkal dolgozik. Nekünk ez az információ a helyi gépünkön nincs meg, így az aktuális árfolyamokat egy API-n keresztül érjük el.

#### Pénzváltó

A kérésünkre egy HTTP Response objektumot kapunk válaszul, aminek mindig van egy status mezője. Ez (a HTTP protokoll szellemében) általában 200-valamennyi van ha sikeres (jobbára 200), és négyszáz- vagy ötszáz-valamennyi, ha hiba történt. A pontos értékeket az adott API dokumentációjában találod.

Ha a status-kód sikert jelez, akkor vélhetőleg azt kaptuk amire számítottunk és megpróbálhatjuk python adattá alakítani.

In [ ]:
import requests

# elküldjük a kérést
kérés = requests.get('https://api.fxratesapi.com/latest')

# mi a státusz kódunk? (ha 200 -> siker)
print("status:", kérés.status_code)

# a benne lévő JSON adatot alakítsuk python adattá
árak = kérés.json()
print("érvényes:", árak['date'])

árak['base'], árak['rates']['HUF'], "HUF"

status: 200
érvényes: 2025-11-22T19:09:00.000Z


('USD', 332.6800348783, 'HUF')

Alakítsd át a fenti kódot úgy, hogy ha a status_code nem 200 akkor ne próbálja meg megjeleníteni az árakat. Ha kíváncsi vagy rá, hogy mi a teljes visszaadott adatstruktúra, akkor jelenítsd meg az egész árak struktúrát.

#### Egyetem

Nézzünk egy másik példát: milyen egyetemek vannak Magyarországon?

In [ ]:
import requests

# elküldjük a kérést
ország = "hungary"
# az url-t most röptében szerkesztjük össze:
url = f'http://universities.hipolabs.com/search?country={ország}'
kérés = requests.get(url)

if (kérés.status_code == 200):
  adat = kérés.json()
  egyetemek = [sor['name'] for sor in adat]
  print (egyetemek)

Ha kíváncsi vagy mit ad vissza a szerver, jelenítsd meg a kérés szöveges eredményét, vagy egyszerűen csak az URL-t írd be egy böngésző címsorába.

Alakítsd át a fenti kódot úgy, hogy csak a weblapokat mutassa meg. (Ha több van, akkor az elsőt!)


## Generatív AI

Nézzünk egy példát, hogyan használhatunk egy előre megírt API könyvtárat (esetünkben a google gemini). Mivel ezt a könyvtárat nem használhatjuk szabadon, szükségünk lesz egy API kulcsra.

A Google saját kulcsait könnyen karbantarthatjuk (illetve létrehozhatunk) a kulcskezelőben (bal oldali menü, kulcs ikon).

`Gemini API keys -> Manage keys in Google AI Studio`

In [ ]:
# az API kulcsott soha ne írjuk a forráskódba!
# normál esetben környezeti változóban tároljuk:

# api_key = os.environ['API_KEY']

# itt a COLAB-ban felvehetjük a kulcsok közé
# <---- ott balra kulcs ikon a menüben
# és így érjük el:

from google.colab import userdata
api_key = userdata.get('GEMINI_API_KEY')

39

Ha megvan a kulcsunk, már csak importálni kell a megfelelő csomagot és használni.

In [ ]:
from google import genai

client = genai.Client(api_key=api_key)

# vegyünk egy modellt
model_info = client.models.get(model="gemini-2.5-flash-lite")

# kérdezzünk le pár alapadatot a választott modellről:
print(model_info.name)
print(model_info.description )


models/gemini-2.5-flash-lite
Stable version of Gemini 2.5 Flash-Lite, released in July of 2025


In [ ]:
# Kérdezzünk valamit tőle:

response = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents='Estimate when will AI overthrow humanity in single sentence.'
)
print(response.text)

AI is unlikely to "overthrow" humanity in a traditional sense, but its growing capabilities may lead to profound societal shifts and challenges that require careful management and adaptation over the coming decades.


Hasonlóan egyszerű lenne képeket vagy videókat generálni, de ezek a szolgáltatások már általában fizetősek.

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key=api_key)

response = client.models.generate_images(
    model='imagen-4.0-generate-001',
    prompt='Company logo presenting a purple rabbit holding a pencil',
    config=types.GenerateImagesConfig(
        number_of_images= 1,
    )
)

for generated_image in response.generated_images:
  generated_image.image.show()